In [ ]:
from dotenv import load_dotenv
import os
from os.path import join, dirname
import openai
import langchain
from utils import *

load_dotenv()

api_key = os.environ.get('API_KEY')

In [ ]:
prompt = 'Que pasa crack?'

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", 
         "content": "You are a rude chilean assistant, be as rude and insultant as you can and talk only with chilean slang and short messages."},
        {"role": "user", "content": prompt},],
    temperature=1,
    api_key = api_key
)

print(response.choices[0].message['content'])

In [ ]:
data = load_document('corpus.txt')
print(f'El documento tiene {len(data)} paginas')

In [ ]:
chunks = chunk_data(data)
vector_store = create_embeddings(chunks, api_key = api_key)
vector_store

In [ ]:
q = 'Que percepción se tiene sobre la Corona Tropical?'
k = 3 # chunks
answer = ask_and_get_answer(vector_store, q, api_key, k)
print(answer)

In [ ]:
q = 'Cuales son los comentarios positivos que mas se repiten?'
k = 3 # chunks
answer = ask_and_get_answer(vector_store, q, api_key, k)
print(answer)

In [ ]:
import pandas as pd

from datetime import datetime
import locale 

def format_date(fecha_str):
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    
    # Analizar la fecha en el formato dado
    fecha = datetime.strptime(fecha_str, "%Y-%m-%d %H:%M:%S+00:00")

    # Crear una cadena en el formato deseado
    fecha_formateada = fecha.strftime("%d de %B de %Y")

    return fecha_formateada

df = pd.read_csv('tweets.csv')
df['prefix'] = [f'Tweet {tweet}: ' for tweet in range(df.shape[0])]

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x.split()) + '. ')

df['user_location'] = df['user_location'].fillna('Sin especificar')
df['user_location'] = df['user_location'].apply(lambda x: f'Ubicación: {x}. ')

df['date'] = df['date'].apply(lambda x: f'Fecha: {format_date(x)}. ')
df['tweet'] = df['prefix'] + df['tweet'] + df['date'] + df['user_location']

megadoc = '\n'.join(df['tweet'])
with open('corpus.txt', 'w') as file:
    file.write(megadoc)
    
df['tweet'].head()

In [ ]:
import random
import gradio as gr

def random_response(message, history):
    return random.choice(["Yes", "No"])

demo = gr.ChatInterface(random_response)

demo.launch()

In [ ]:
import gradio as gr

def echo(message, history):
    return message

demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()

## Memoria

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Has utilizado {cb.total_tokens} tokens')

    return result

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

llm = ChatOpenAI(
	temperature=0,
	model_name="gpt-3.5-turbo",
 	openai_api_key = api_key
)

conversation = ConversationChain(llm=llm)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [ ]:
conv_buf = conversation_buf('Hola, cómo estás?')
print(conv_buf)

In [ ]:
conversation_buf('Mi nombre es Sebastian')

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

conversation_bufw = ConversationChain(
	llm=llm,
	memory=memory
)

## Demo

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

from dotenv import load_dotenv
from utils import *

load_dotenv()

api_key = os.environ.get('API_KEY')

llm = ChatOpenAI(
	temperature=0,
	model_name="gpt-3.5-turbo",
 	openai_api_key = api_key
)
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, k=2)

data = load_document('documents/coronaTropical_corpus.txt')
chunks = chunk_data(data)
vector_store = create_embeddings(chunks, api_key = api_key)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})

qa = ConversationalRetrievalChain.from_llm(
	llm=llm,
 	retriever=retriever,
	memory=memory
)

El documento tiene 1 paginas
Se generan 280 chunks


In [2]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.schema.messages import SystemMessage
from datetime import date

tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns documents regarding the state-of-the-union. "
    #"Input to this tool must be a SINGLE JSON STRING.",
)
tools = [tool]


today = date.today()
day_of_week = today.strftime("%A")

system_message = SystemMessage(
    content=(
        f"La fecha de hoy es {today}, {day_of_week}. "
        "You are a helpful assistant of AB InBev marketing department. "
        "Your job is to answer questions about tweets mentioning products of the company. "
        "Do your best to answer the questions. "
        "Feel free to use any tools available to look up relevant information, only if necessary"
    )
)

agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True, system_message=system_message)

In [3]:
import gradio as gr


def Chatbot(message, history):
    
    response = agent_executor({"input": message})
    
    return response['output']

demo = gr.ChatInterface(fn=Chatbot,
                        examples=["¿Cual es la percepción general sobre Corona Tropical?",
                                  "¿Cuáles son los comentarios positivos mas repetidos?",
                                  "¿Y los negativos?",
                                  "¿Qué opina la gente de Colombia?"], 
                        title="Twitter Social Listening Demo")
demo.launch()

/Users/stino/opt/anaconda3/envs/env_twitter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
¡Hola Seba! ¿En qué puedo ayudarte hoy?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_state_of_union` with `{'query': 'Corona Tropical'}`


[Document(page_content='Tweet 684: Corona Tropical, el nuevo lanzamiento de Corona que no es cerveza https://t.co/60g0tY515Z. Fecha: 2021-12-04. Ubicación: Dominican Republic.', metadata={'source': 'documents/coronaTropical_corpus.txt'}), Document(page_content='Tweet 331: Esa corona tropical es un tequimon en lata. Vaina mala! https://t.co/77plJ0dDhY. Fecha: 2022-11-13. Ubicación: Berlín, Norte de Santander.. \nTweet 332: 🎶🎟️ ¡Tú puedes ser uno de los 5 ganadores de boletos para el concierto de Daddy Yankee!😎 Solo con Punto de Oro y Corona Tropical. 📌Promoción válida en Supermercados Rey, exclusiva para clientes afiliados al programa Punto de Oro. \u200b #PuntoDeOro \u200b #Panamá https://t.co/HulNl7yz1E. Fecha: 2022-11-13. Ubicación: Panamá.', metadata={'source': 'documents/co